## Filter Python database to find better, cheaper players using FIFA’s ratings

produit par : [FC Python](https://fcpython.com/blog/filter-python-database-to-find-better-cheaper-players-using-fifas-ratings)

données issues de : [Kaggle](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset#players_20.csv)

In [1]:
#Import modules
import numpy as np
import pandas as pd 

In [2]:
# chargement des données
data = pd.read_csv("fifa-20-complete-player-dataset/players_20.csv")
data.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,...,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


In [5]:
#Define function called club, that looks for a team name and return the selected columns
def club(teamName):
    
    return data[data['club'] == teamName][['short_name','wage_eur','value_eur','player_positions','overall','age']]


#Use the club function to find the team, and sort the squad by wage bill
club('Manchester United').sort_values("wage_eur", ascending = False)


,short_name,wage_eur,value_eur,player_positions,overall,age
2526,R. Amalfitano,14000,5500000,"CM, CDM",74,29
2474,B. Ecuélé Manga,12000,4600000,"CB, RB",74,30
2706,Mama Baldé,12000,6000000,"RW, RM, RB",73,23
3934,Júlio Tavares,12000,3000000,ST,72,30
3660,J. Marié,11000,3500000,"CM, CDM",72,27
3448,D. Ndong,10000,3700000,"CDM, CM",72,25
3876,W. Lautoa,10000,2300000,CB,72,31
3933,F. Chafik,10000,1900000,"RB, RWB, LB",72,32
5162,B. Jeannot,9000,2100000,"ST, RW",70,27
5375,F. Sammaritano,9000,1100000,"LM, LW, RM",70,33


In [9]:
#Extract DDG's information, just like we did with the team name before
DDG = data[data['short_name'] == 'De Gea'][['short_name','wage_eur','value_eur','player_positions','overall','age']]

#Assign DDG's wage, position, rating and age to variables
DDGWage = DDG['wage_eur'].item()
DDGPos = DDG['player_positions'].item()
DDGRating = DDG['overall'].item()
DDGAge = DDG['age'].item()


,short_name,wage_eur,value_eur,player_positions,overall,age
14,De Gea,205000,56000000,GK,89,28


In [10]:
#Create a list of goalkeepers, matching DDG's position
longlist = data[data['player_positions'] == DDGPos][['short_name','wage_eur','value_eur','player_positions','overall','age']]

#Create a list of players that have a lower overall rank than DDG
removals = longlist[longlist['overall'] <= DDGRating].index
#Drop these players
longlist.drop(removals , inplace=True)

#Repeat above, but for players with a larger wage
removals = longlist[longlist['wage_eur'] > DDGWage].index
longlist.drop(removals , inplace=True)

#Repeat above, but for older players
removals = longlist[longlist['age'] >= DDGAge].index
longlist.drop(removals , inplace=True)

#Show me our potential replacements, sorted by lowest wages
longlist.sort_values("wage_eur")

,short_name,wage_eur,value_eur,player_positions,overall,age
3,J. Oblak,125000,77500000,GK,91,26


In [11]:


def cheapReplacement(player, skillReduction = 0):
    
    #Get the replacee with the name provided in the argument
    replacee = data[data['short_name'] == player][['short_name','wage_eur','value_eur','player_positions','overall','age']]

    #Assign the relevant details of this player to variables
    replaceePos = replacee['player_positions'].item()
    replaceeWage = replacee['wage_eur'].item()
    replaceeAge = replacee['age'].item()
    replaceeOverall = replacee['overall'].item() - skillReduction
    
    #Create the longlist of players that share the position
    longlist = data[data['player_positions'] == replaceePos][['short_name','wage_eur','value_eur','player_positions','overall','age']]
    
    
    #Create list of players that do not meet the rating criteria and drop them from the longlist
    removals = longlist[longlist['overall'] <= replaceeOverall].index
    longlist.drop(removals , inplace=True)
    
    #Repeat for players with higher wages
    removals = longlist[longlist['wage_eur'] > replaceeWage].index
    longlist.drop(removals , inplace=True)

    #Repeat for older players
    removals = longlist[longlist['age'] >= replaceeAge].index
    longlist.drop(removals , inplace=True)
    
    #Display the players that meet the requirements
    return longlist.sort_values("wage_eur")


* Remplacement abordable de Fred

In [12]:
cheapReplacement('Fred')

,short_name,wage_eur,value_eur,player_positions,overall,age
330,H. Aouar,59000,26000000,CM,81,21
259,K. Demirbay,75000,27500000,CM,82,25
245,Rúben Neves,89000,29000000,CM,82,22


* Remplacement abordable de Lindelöf

In [14]:
cheapReplacement('V. Lindelöf')

,short_name,wage_eur,value_eur,player_positions,overall,age
165,J. Tah,67000,31000000,CB,83,23
246,P. Kimpembe,75000,26500000,CB,82,23
73,M. de Ligt,76000,50000000,CB,85,19
170,M. Akanji,77000,31000000,CB,83,23
177,D. Sánchez,93000,30500000,CB,83,23


* Remplacement abdordable de Pogba avec une circonférence de 8 

In [22]:
cheapReplacement('P. Pogba', 8)

,short_name,wage_eur,value_eur,player_positions,overall,age
331,T. Ndombele,87000,26000000,"CM, CDM",81,22
243,N. Keïta,95000,29000000,"CM, CDM",82,24
166,C. Tolisso,110000,34000000,"CM, CDM",83,24
338,E. Can,110000,23000000,"CM, CDM",81,25
173,A. Rabiot,120000,33000000,"CM, CDM",83,24
75,F. de Jong,195000,52000000,"CM, CDM",85,22
